# Rasterize MT drought monitor shapefiles
This notebook takes clipped MT drought monitor shapefiles and rasterizes and saves as a netcdf file.

In [ ]:
import geopandas as gpd
import xarray as xr
from geocube.api.core import make_geocube
import glob
import datetime

In [ ]:
dir = '/media/nick/Seagate Backup Plus Drive/Data/Drought_Monitor/Montana_Clipped/'
zips = sorted(glob.glob(dir + '*.zip'))

In [ ]:
full_cube = None
for zip in zips:
    time_str = zip.split('/')[-1].split('_')[1] 
    tstamp = datetime.datetime.strptime(time_str, '%Y%m%d')
    gdf = gpd.read_file(zip)
    cube = make_geocube(
        gdf,
        measurements=['DM'],
        resolution=(4000, -4000),
    )
    cube = cube.assign_coords(time = tstamp)
    if not full_cube:
        full_cube = cube
    else:
        full_cube = xr.concat([full_cube, cube], 'time')
    print("*******************************************")
    print("The date " + time_str + " has been added to xarray")

## Save as netcdf

In [ ]:
full_cube.to_netcdf(dir + 'All_Droughts_MT.nc')

## Open netcdf as xarray dataset and save to geotiff

In [ ]:
ds = xr.open_dataset(dir + 'All_Droughts_MT.nc').fillna(-9999)
ds.DM.rio.to_raster(dir + 'All_Droughts_MT.tif')

## Save dates of netcdf as text file

In [ ]:
import numpy as np
np.savetxt('time.txt', ds.time, fmt="%s", delimiter=',')